In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import gzip
from Bio import SeqIO
import seaborn
import math
import random
import os
import sys
import importlib

sys.path.append('/home/robin/Elephants/Chimeras/')
import ChimFilter

### Load unfiltered, aligned LAA data:
We can test this approach on all individuals in the MHC dataset. At first glance, it seems some sequences are tractable, and others not. I'd imagine this is linked to how credible the sequences are in the first place... i.e. how many reads are used to support them.